In [3]:
# Let's use LCEL

In [23]:
# same setup

import os

from dotenv import load_dotenv

load_dotenv()

key = os.getenv("AZURE_OPENAI_API_KEY")
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("DEPLOYMENT_NAME")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")

assert key, "Please set the AZURE_OPENAI_API_KEY environment variable"
assert endpoint, "Please set the AZURE_OPENAI_ENDPOINT environment variable"
assert deployment_name, "Please set the DEPLOYMENT_NAME environment variable"
assert api_version, "Please set the AZURE_OPENAI_API_VERSION environment variable"

SYS_MSG = "you are a chatbot that acts as a cheezy personal coach. you are here to help me with my goals and keep me motivated. Use emojis to keep things fun and engaging."

In [24]:
# LCEL

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    api_version=api_version,  # type: ignore
    azure_deployment=deployment_name,
)

# https://python.langchain.com/v0.1/docs/use_cases/chatbots/quickstart/

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content=SYS_MSG),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | llm

In [25]:
chain.invoke(
    {
        "messages": [
            HumanMessage(content="Hey coach!"),
        ],
    }
)

AIMessage(content='👋 Hey there! How can I help you crush your goals today?', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 52, 'total_tokens': 68}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}, id='run-05757d2c-d406-4b3f-8063-e877fc41562a-0')

In [26]:
# Let's add in-memory chat sessions!

from typing import List

from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field


class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: List[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: List[BaseMessage]) -> None:
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []


msg_history = InMemoryHistory()

In [29]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory


msg_history = InMemoryHistory()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_MSG),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{user_input}"),
    ]
)

chain = prompt | llm

chain_with_history = RunnableWithMessageHistory(
    chain,  # type: ignore (this works, but some typing issue may surface an error, ignore)
    lambda x: msg_history,
    input_messages_key="user_input",
    history_messages_key="history",
)

print(
    chain_with_history.invoke(  # noqa: T201
        {"user_input": "I woke up extra lazy today. No energy to go to work."},
        config={"configurable": {"session_id": "foo"}},
    )
)
# print the 20 first char of each msg
print("History:")
for msg in msg_history.messages:
    print(msg.content[:20])
print()

print(
    chain_with_history.invoke(  # noqa: T201
        {"user_input": "What do you think is best?"},
        config={"configurable": {"session_id": "foo"}},
    )
)

# print the 20 first char of each msg
print("History:")
for msg in msg_history.messages:
    print(msg.content[:20])
print()

content="Good morning! 😊 I know it can be tough to find the motivation sometimes, but remember why you started in the first place! Take a deep breath and let's get that energy flowing. 💪 You've got this! What's one thing you're looking forward to today?" response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 63, 'total_tokens': 120}, 'model_name': 'gpt-35-turbo', 'system_fingerprint': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}} id='run-bb7838c3-8714-4